<a href="https://colab.research.google.com/github/zeev-haydar/ML-TuneHive/blob/model-development%2Fhaidar/model-dev/preprocessing-modeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Preprocessing and Modeling

## Import required modules

In [1]:
import tensorflow as tf
from sklearn.model_selection import train_test_split
import tensorflow_datasets as tfds
from keras.api.layers import Dense, Embedding, GRU, LeakyReLU, Concatenate, Masking, Layer, StringLookup, Normalization, BatchNormalization
from keras.api import Input
from keras.api.models import Model
from keras.api.losses import SparseCategoricalCrossentropy
from keras.api.metrics import SparseCategoricalAccuracy, Mean, TopKCategoricalAccuracy
# from transformers.models.bert import TFBertTokenizer, TFBertEmbeddings  # embedding and tokenizer for description/nlp related stufff
from keras.api.optimizers import Adam
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import ast

## Preprocessing

### Load CSV

In [2]:
# For Running in Google Colab
# "https://raw.githubusercontent.com/{user}/{repo}/main/{src_dir}/{file}"
url = "https://raw.githubusercontent.com/zeev-haydar/ML-TuneHive/main/model-dev/data/session-data.csv"
!wget --no-cache --backups=1 {url}

'wget' is not recognized as an internal or external command,
operable program or batch file.


In [122]:
import keras
import os
print(keras.__version__)

# root_path = "data"
root_path = "" # if using colab
df_ori = pd.read_csv(os.path.join(root_path, "session-data.csv"))
# Ambil 10.000 baris secara acak
df = df_ori.sample(n=10000, random_state=42)

df

3.7.0


,index_x,SongID,TimeStamp_Central,Performer_x,Album,Song_x,TimeStamp_UTC,index_y,Performer_y,Song_y,...,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,session_3_hour,session_id
49638,62481,The ManAloe Blacc,1/12/2017 12:58:00 PM,Aloe Blacc,Lift Your Spirit,The Man,2017-01-12 18:58:00,1380,Aloe Blacc,The Man,...,0.0,0.0650,0.033100,0.000000,0.2140,0.488,81.853,4.0,2017-01-12 18:00:00,33
30837,41564,Let the Rhythm JustThe Polish Ambassador,10/29/2018 4:14:00 PM,The Polish Ambassador,Pushing Through the Pavement,Let the Rhythm Just,2018-10-29 21:14:00,788,The Polish Ambassador,Let the Rhythm Just,...,1.0,0.0748,0.309000,0.000197,0.0779,0.785,98.009,4.0,2018-10-29 21:00:00,1443
41389,53305,AstronomyMetallica,12/5/2017 8:06:00 PM,Metallica,Garage Inc.,Astronomy,2017-12-06 02:06:00,12821,Metallica,Astronomy,...,0.0,0.0413,0.000157,0.010600,0.1460,0.157,85.741,4.0,2017-12-06 00:00:00,679
19454,28812,Katie Queen Of TennesseeThe Apache Relay,7/9/2019 7:22:00 AM,The Apache Relay,Apache Relay,Katie Queen Of Tennessee,2019-07-09 12:22:00,5475,The Apache Relay,Katie Queen Of Tennessee,...,1.0,0.0308,0.100000,0.000252,0.2210,0.729,127.024,4.0,2019-07-09 12:00:00,2262
38888,50540,Tell MeBros,2/12/2018 3:37:00 PM,Bros,Tell Me,Tell Me,2018-02-12 21:37:00,8675,Bros,Tell Me,...,0.0,0.0359,0.199000,0.000277,0.1080,0.944,100.078,4.0,2018-02-12 21:00:00,826
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9233,15973,Lord of the FriesPolyrhythmics,6/8/2020 11:51:00 AM,Polyrhythmics,Caldera,Lord of the Fries,2020-06-08 17:51:00,12990,Polyrhythmics,Lord of the Fries,...,0.0,0.0501,0.156000,0.916000,0.0804,0.634,122.131,4.0,2020-06-08 15:00:00,3184
42280,54309,RegulateWarren G,11/11/2017 9:08:00 AM,Warren G,Regulate… G Funk Era (Special Edition),Regulate,2017-11-11 15:08:00,3378,Warren G,Regulate,...,0.0,0.2160,0.306000,0.000002,0.1150,0.763,95.262,4.0,2017-11-11 15:00:00,618
33427,44432,EuphoriaLucy Vincent,8/3/2018 3:07:00 PM,Lucy Vincent,Euphoria,Euphoria,2018-08-03 20:07:00,10759,Lucy Vincent,Euphoria,...,0.0,0.0418,0.990000,0.928000,0.0931,0.106,82.017,4.0,2018-08-03 18:00:00,1234
15888,25022,Imaginary LoverAtlanta Rhythm Section,11/2/2019 4:32:00 PM,Atlanta Rhythm Section,20th Century Masters: The Millennium Collectio...,Imaginary Lover,2019-11-02 21:32:00,8509,Atlanta Rhythm Section,Imaginary Lover,...,0.0,0.0259,0.383000,0.548000,0.1280,0.413,109.499,4.0,2019-11-02 21:00:00,2541


In [123]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10000 entries, 49638 to 14237
Data columns (total 30 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   index_x                    10000 non-null  int64  
 1   SongID                     10000 non-null  object 
 2   TimeStamp_Central          10000 non-null  object 
 3   Performer_x                10000 non-null  object 
 4   Album                      9554 non-null   object 
 5   Song_x                     10000 non-null  object 
 6   TimeStamp_UTC              10000 non-null  object 
 7   index_y                    10000 non-null  int64  
 8   Performer_y                10000 non-null  object 
 9   Song_y                     10000 non-null  object 
 10  spotify_genre              10000 non-null  object 
 11  spotify_track_id           10000 non-null  object 
 12  spotify_track_preview_url  7230 non-null   object 
 13  spotify_track_duration_ms  10000 non-null  floa

In [124]:
test_col_name = 'mode'
df.loc[:, test_col_name]

49638    0.0
30837    1.0
41389    0.0
19454    1.0
38888    0.0
        ... 
9233     0.0
42280    0.0
33427    0.0
15888    0.0
14237    0.0
Name: mode, Length: 10000, dtype: float64

## Remove N.A.N data

In [125]:
# df_filtered = df[~df['danceability'].isna()]
# df_filtered.info()

### Prepare Tensorflow Datasets

In [126]:
import time
from sklearn.preprocessing import LabelEncoder


from sklearn.model_selection import train_test_split

# Feature columns (as provided)
feature_columns = [
    'spotify_genre',
    'spotify_track_popularity',
    'danceability',
    'loudness',
    'tempo',
]

# Define the DataPreprocessor class
class DataPreprocessor:
    def __init__(self, df, feature_columns, batch_size=32, fixed_genre_size=10, train_size=0.8):
        """
        Initializes the data preprocessor with necessary parameters and preprocessing layers.

        Args:
            df (DataFrame): The input DataFrame containing session data.
            feature_columns (list): List of feature column names.
            batch_size (int): The batch size for dataset creation.
            fixed_genre_size (int): The fixed size for genre vectorization.
            train_size (float): Proportion of the data to use for training (between 0 and 1).
        """
        self.df = df
        self.feature_columns = feature_columns
        self.batch_size = batch_size
        self.fixed_genre_size = fixed_genre_size
        self.train_size = train_size

        # Split the dataset into training and testing datasets
        self.train_df, self.test_df = train_test_split(self.df, train_size=self.train_size, random_state=42)

        # Numeric feature preprocessing
        self.numeric_data = self.df[feature_columns[1:]].apply(pd.to_numeric, errors='coerce')
        self.mean_values = self.numeric_data.mean()
        self.std_values = self.numeric_data.std()

        # Initialize LabelEncoder for SongID and spotify_genre
        self.song_id_encoder = LabelEncoder()
        self.genre_encoder = LabelEncoder()

        # Extract unique SongIDs and genres
        unique_song_ids = self.df['SongID'].unique()
        all_genres = []
        for genre_str in self.df['spotify_genre']:
            try:
                genre_list = ast.literal_eval(genre_str)  # Safely parse the string into a list
                if isinstance(genre_list, list):
                    all_genres.extend(genre_list)
            except Exception as e:
                print(f"Error parsing genre: {e}")

        unique_genres = list(set(all_genres))

        # Fit the LabelEncoders on the data
        self.song_id_encoder.fit(unique_song_ids)
        self.genre_encoder.fit(unique_genres)

        self.items_size = len(self.song_id_encoder.classes_)  # Number of unique SongIDs
        self.genres_size = len(self.genre_encoder.classes_)
        print(self.items_size,self.genres_size)
        self.dataset = None

    def preprocess_song_id(self, song_id):
        """
        Encode the SongID using LabelEncoder.
        """
        return self.song_id_encoder.transform([song_id])[0]

    def clean_genre(self, value, default_value=0, dtype=tf.int32):
        """
        Clean and process the 'spotify_genre' feature.
        """
        if value is None or (isinstance(value, str) and not value.strip()):
            return np.full((self.fixed_genre_size,), default_value, dtype=dtype.as_numpy_dtype)

        try:
            genre_list = eval(value) if isinstance(value, str) else value
            if isinstance(genre_list, list):
                genre_encoded = self.genre_encoder.transform(genre_list)
            else:
                genre_encoded = self.genre_encoder.transform([value])
        except Exception:
            genre_encoded = self.genre_encoder.transform([value])

        # Pad or truncate to fixed size
        return np.pad(genre_encoded, (0, max(0, self.fixed_genre_size - len(genre_encoded))),
                      mode='constant')[:self.fixed_genre_size].astype(dtype.as_numpy_dtype)

    def clean_numeric_feature(self, value, default_value=0.0, feature_name="feature", mean=None, std=None):
        """
        Clean, process, and normalize numerical features using Z-score normalization.
        """
        if value is None or (isinstance(value, float) and np.isnan(value)):
            return default_value

        try:
            value = float(value)
            # Apply Z-score normalization if mean and std are provided
            if mean is not None and std is not None and std != 0:
                z_score_value = (value - mean) / std
                return z_score_value
            return value  # Return raw value if no normalization
        except ValueError:
            return default_value

    def create_session_dataset(self, session_df):
        """
        Create session dataset as a list of dictionaries for each session.
        """
        session_df = session_df.sort_values(by=['session_id', 'TimeStamp_UTC'])
        grouped = session_df.groupby('session_id')
        sessions_data = []
        for session_id, group in grouped:
            session_data = group.to_dict(orient='records')
            sessions_data.append(session_data)
        return sessions_data

    def preprocess_data(self, sessions, k=1):
        item_sequences = []
        next_item_sequences = []
        genre_sequences = []
        next_genre_sequences = []
        feature_sequences = []
        processed_item_count = 0
    
        for idx, session in enumerate(sessions):
            session_item_sequences = []
            session_next_item_sequences = []
            session_genre_sequences = []
            session_next_genre_sequences = []
            session_feature_sequences = []
    
            for i in range(len(session) - 1):
                session_item_encoded = self.preprocess_song_id(session[i]['SongID'])
                next_session_item_encoded = self.preprocess_song_id(session[i + 1]['SongID'])
                session_item_sequences.append(session_item_encoded)
                session_next_item_sequences.append(next_session_item_encoded)
    
                genre_cleaned = self.clean_genre(session[i].get('spotify_genre', None))
                next_genre_cleaned = self.clean_genre(session[i + 1].get('spotify_genre', None))
                session_genre_sequences.append(genre_cleaned)
                session_next_genre_sequences.append(next_genre_cleaned)
    
                numeric_features = []
                for col in self.feature_columns:
                    if col != 'spotify_genre':
                        mean = self.mean_values.get(col, None)
                        std = self.std_values.get(col, None)
                        cleaned_feature = self.clean_numeric_feature(session[i].get(col, None), mean=mean, std=std)
                        numeric_features.append(cleaned_feature)
    
                session_feature_sequences.append(numeric_features)
    
            if len(session_next_item_sequences) > k:
                item_sequences.extend(session_item_sequences)
                next_item_sequences.extend(session_next_item_sequences)
                genre_sequences.extend(session_genre_sequences)
                next_genre_sequences.extend(session_next_genre_sequences)
                feature_sequences.extend(session_feature_sequences)
                processed_item_count += len(session_item_sequences)
                print(f"Session {idx + 1} processed with {len(session_item_sequences)} items.")
            else:
                print(f"Session {idx + 1} skipped because next item sequence length is {len(session_next_item_sequences)}.")
    
        print(f"Total processed items: {processed_item_count}")
    
        item_sequences = tf.convert_to_tensor(item_sequences, dtype=tf.int32)
        next_item_sequences = tf.convert_to_tensor(next_item_sequences, dtype=tf.int32)
        genre_sequences_tensor = tf.convert_to_tensor(genre_sequences, dtype=tf.int32)
        next_genre_sequences = tf.convert_to_tensor(next_genre_sequences, dtype=tf.int32)
        feature_sequences_tensor = tf.convert_to_tensor(feature_sequences, dtype=tf.float32)
    
        dataset = tf.data.Dataset.from_tensor_slices({
            'item': item_sequences,
            'genre': genre_sequences_tensor,
            'features': feature_sequences_tensor,
            'next_item': next_item_sequences,
            'next_genre': next_genre_sequences
        })
    
        return dataset


    def create_session_dataset_tensor(self, k=1):
        """
        Main function to create session dataset as tensors and return the dataset.
        """
        if self.dataset is not None:
            print("Dataset already created")
            return

        print("Creating session dataset")
        sessions_data = self.create_session_dataset(self.train_df)  # Use train data for training
        print("Creating tensor dataset")
        dataset = self.preprocess_data(sessions_data, k=k)

        # Shuffle and batch the training data with buffer size adjusted for performance
        dataset = (
            dataset.shuffle(buffer_size=1024)  # Adjusted buffer size (can be increased depending on memory)
                   .batch(self.batch_size, drop_remainder=True)
                   .prefetch(buffer_size=tf.data.AUTOTUNE)  # Prefetch with automatic optimization
        )

        self.dataset = dataset
        return dataset

    def get_test_data(self, k):
        """
        Return preprocessed test dataset without shuffling.
        """
        sessions_data = self.create_session_dataset(self.test_df)
        dataset = self.preprocess_data(sessions_data, k)

        # Batch the test data without shuffling
        dataset = (
            dataset.batch(self.batch_size, drop_remainder=True)
        )

        return dataset


In [127]:
preprocessor = DataPreprocessor(df, feature_columns)

# Create the session dataset tensor
train_dataset = preprocessor.create_session_dataset_tensor()


4283 845
Creating session dataset
Creating tensor dataset
Session 1 skipped because next item sequence length is 1.
Session 2 skipped because next item sequence length is 0.
Session 3 skipped because next item sequence length is 1.
Session 4 skipped because next item sequence length is 0.
Session 5 skipped because next item sequence length is 0.
Session 6 processed with 5 items.
Session 7 processed with 3 items.
Session 8 skipped because next item sequence length is 0.
Session 9 skipped because next item sequence length is 0.
Session 10 processed with 3 items.
Session 11 skipped because next item sequence length is 0.
Session 12 processed with 2 items.
Session 13 skipped because next item sequence length is 0.
Session 14 skipped because next item sequence length is 0.
Session 15 skipped because next item sequence length is 0.
Session 16 processed with 2 items.
Session 17 processed with 2 items.
Session 18 skipped because next item sequence length is 0.
Session 19 processed with 2 items

In [128]:
test_dataset = preprocessor.get_test_data(k=1)

Session 1 skipped because next item sequence length is 1.
Session 2 skipped because next item sequence length is 1.
Session 3 skipped because next item sequence length is 1.
Session 4 skipped because next item sequence length is 0.
Session 5 skipped because next item sequence length is 1.
Session 6 skipped because next item sequence length is 0.
Session 7 skipped because next item sequence length is 0.
Session 8 skipped because next item sequence length is 1.
Session 9 skipped because next item sequence length is 1.
Session 10 skipped because next item sequence length is 0.
Session 11 skipped because next item sequence length is 0.
Session 12 skipped because next item sequence length is 0.
Session 13 skipped because next item sequence length is 0.
Session 14 skipped because next item sequence length is 0.
Session 15 skipped because next item sequence length is 1.
Session 16 skipped because next item sequence length is 0.
Session 17 skipped because next item sequence length is 0.
Sessio

In [129]:
for batch in train_dataset.take(1):
    print("Items (SongID):", batch['item'].numpy())
    print("Genre:", batch['genre'].numpy())
    print("Features:", batch['features'].numpy())
    print("Next Items (Next SongID):", batch['next_item'].numpy())

for batch in test_dataset.take(1):
    print("Items (SongID):", batch['item'].numpy())
    print("Genre:", batch['genre'].numpy())
    print("Features:", batch['features'].numpy())
    print("Next Items (Next SongID):", batch['next_item'].numpy())

Items (SongID): [1706 4075  252 1564  327  812 3796 3437 4260 3984 1481 1945 1573 4077
 1047 3160 1212 2190 1674  336 1055 3557 3882 1425  564 1436 1033 1436
  487 2985 3507  115]
Genre: [[252   0   0   0   0   0   0   0   0   0]
 [275 507 515 620   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [ 12   0   0   0   0   0   0   0   0   0]
 [ 17 314 575 632 664 680   0   0   0   0]
 [ 17 314 575 632 664 680   0   0   0   0]
 [ 17 352 491 592 680 740 778   0   0   0]
 [212 620 626 638 816   0   0   0   0   0]
 [234 483 662   0   0   0   0   0   0   0]
 [515   0   0   0   0   0   0   0   0   0]
 [104 490 651 680   0   0   0   0   0   0]
 [507 515   0   0   0   0   0   0   0   0]
 [ 10 331 352 491 639 645 680 740   0   0]
 [418   0   0   0   0   0   0   0   0   0]
 [  7  98 300 469 681 682   0   0   0   0]
 [212 620 627 638   0   0   0   0   0   0]
 [212 620 626 660   0   0   0   0   0   0]
 [212 530 620 627 638 773   0   0   0   0]
 [212 620 638   0   0   0   0   0   0  

## Define Model class

In [137]:
class ItemEmbedding(Layer):
    def __init__(self, num_items, item_embed_dim):
        super(ItemEmbedding, self).__init__()
        self.item_embedding = Embedding(input_dim=num_items, output_dim=item_embed_dim, mask_zero=True)

    def call(self, items):
        return self.item_embedding(items)


class GRU4REC(Model):
    def __init__(self, rnn_params, genre_embed_dim, item_embed_dim, ffn1_units, feature_dense_units, preprocessed_data):
        super(GRU4REC, self).__init__()
        self.embedding = ItemEmbedding(preprocessed_data.items_size, item_embed_dim)
        self.genre_embedding = Embedding(input_dim=preprocessed_data.genres_size, output_dim=genre_embed_dim, mask_zero=True)

        self.rnn_layers = [GRU(**rnn_params[0], return_sequences=True)]
        for i in range(1, len(rnn_params) - 1):
            self.rnn_layers.append(GRU(**rnn_params[i], return_sequences=True))
        self.rnn_layers.append(GRU(**rnn_params[-1], return_sequences=False))

        self.concat = Concatenate(axis=-1)
        self.batch_norm = BatchNormalization()

        self.feature_dense = Dense(feature_dense_units, activation='relu')
        self.ffn1 = Dense(ffn1_units)
        self.activation1 = LeakyReLU(alpha=0.2)

        self.item_output = Dense(preprocessed_data.items_size, activation='softmax', name='item_output')
        self.genre_output = Dense(preprocessed_data.genres_size, activation='softmax', name='genre_output')

    def call(self, inputs, training=False):
        item_sequences, item_features, item_genres = inputs

        item_embedded = self.embedding(item_sequences)
        item_embedded = tf.expand_dims(item_embedded, axis=1)

        genre_embedded = self.genre_embedding(item_genres)
        genre_embedded = tf.reduce_mean(genre_embedded, axis=1)
        genre_embedded = tf.expand_dims(genre_embedded, axis=1)

        feature_transformed = self.feature_dense(item_features)
        feature_transformed = tf.expand_dims(feature_transformed, axis=1)

        combined_input = tf.concat([item_embedded, feature_transformed, genre_embedded], axis=-1)
        combined_input = self.batch_norm(combined_input)

        x = combined_input
        for i in range(len(self.rnn_layers)):
            x = self.concat([combined_input, x]) if i > 0 else x
            x = self.rnn_layers[i](x)

        x = self.ffn1(x)
        x = self.activation1(x)
        item_logits = self.genre_output(x)
        genre_logits = self.item_output(x)

        return item_logits, genre_logits


## Training Loop

In [138]:
from keras.api.metrics import Recall

class RecallAtK(tf.keras.metrics.Metric):
    def __init__(self, k=10, name="recall_at_k", **kwargs):
        super(RecallAtK, self).__init__(name=name, **kwargs)
        self.k = k
        self.recall_at_k = Recall(top_k=self.k)

    def update_state(self, y_true, y_pred, sample_weight=None):
        """
        Update the state of the metric.
        """
        # Since y_true is a list of true items and y_pred are the predicted scores,
        # we need to calculate recall for top-k predicted items
        y_true = tf.cast(y_true, tf.int32)

        # Calculate the top-k predicted items
        top_k_preds = tf.argsort(y_pred, axis=-1, direction='DESCENDING')[:, :self.k]

        # Calculate recall by comparing true labels with the top-k predictions
        recall = tf.reduce_mean(tf.cast(tf.equal(y_true, top_k_preds), tf.float32), axis=-1)
        return recall

    def result(self):
        return self.recall_at_k.result()

    def reset_state(self):
        self.recall_at_k.reset_state()

@tf.function
def train_step(batch):
    with tf.GradientTape() as tape:
        predicted_items, logits = model((batch['item'], batch['features'], batch['genre']), training=True)
        loss = loss_fn(batch['next_item'], logits)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    return loss

def train_gru4rec(model, train_dataset, optimizer, loss_fn, epochs, k, val_dataset=None):
    # metric = tf.keras.metrics.TopKCategoricalAccuracy(k=k)

    for epoch in range(epochs):
        print(f"Epoch {epoch + 1}/{epochs}")
        epoch_loss = 0.0

        for step, batch in enumerate(train_dataset):
            loss = train_step(batch)
            if loss is None:
              print(f"Warning: train_step returned None at step {step}")
              continue
            epoch_loss += loss.numpy()

            # Update metric
            # metric.update_state(batch['next_item'], logits)

        print(f"Training Loss: {epoch_loss / (step + 1):.4f}")
        # metric.reset_state()

        if val_dataset:
            val_loss = 0.0
            for step, batch in enumerate(val_dataset):
                _, logits = model((batch['item'], batch['features'], batch['genre']), training=False)
                val_loss += loss_fn(batch['next_item'], logits).numpy()
            print(f"Validation Loss: {val_loss / (step + 1):.4f}")

def plot_training_history(loss_history, metric_history, metric_name, top_k):
    """Plot the training loss and accuracy."""
    epochs = range(1, len(loss_history) + 1)

    # Create subplots for loss and accuracy
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10, 8))

    # Plot the training loss
    ax1.plot(epochs, loss_history, label='Loss', color='blue', linestyle='-', marker='o')
    ax1.set_title('Training Loss')
    ax1.set_xlabel('Epochs')
    ax1.set_ylabel('Loss')
    ax1.legend()

    # Plot the top-k accuracy
    ax2.plot(epochs, metric_history, label=metric_name, color='green', linestyle='-', marker='o')
    ax2.set_title(f'Training {metric_name}')
    ax2.set_xlabel('Epochs')
    ax2.set_ylabel(metric_name)
    ax2.legend()

    plt.tight_layout()
    plt.show()

## Run the training process

In [139]:
# Define the model
# num_items = len()
# feature_vocab_size = len(feature_columns)
def recsys_loss(y_true_items, y_pred_items, y_true_genres, y_pred_genres):
  item_loss = keras.api.losses.sparse_categorical_crossentropy(y_true_items, y_pred_items)
  genre_loss = keras.api.losses.sparse_categorical_crossentropy(y_true_genres, y_pred_genres)
  overall_loss = tf.reduce_mean(item_loss + genre_loss)
  return overall_loss

model = GRU4REC(
    rnn_params=[
        {"units": 128},
        {"units": 128},
        {"units": 64}
    ],
    item_embed_dim=8,
    genre_embed_dim=8,
    ffn1_units=256,
    feature_dense_units=16,
    preprocessed_data=preprocessor
)
print(f"Item Output Layer Size: {model.item_output.units}")
print(f"Genre Output Layer Size: {model.genre_output.units}")

# print(model.trainable_variables)


Item Output Layer Size: 4283
Genre Output Layer Size: 845


In [141]:


optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy()

# Train the model
train_gru4rec(model=model, train_dataset=train_dataset,optimizer=optimizer, loss_fn=loss_fn, epochs=300, k=8)

Epoch 1/300
Training Loss: 1.0918
Epoch 2/300
Training Loss: 1.0909
Epoch 3/300
Training Loss: 1.0820
Epoch 4/300
Training Loss: 1.0697
Epoch 5/300
Training Loss: 1.0741
Epoch 6/300
Training Loss: 1.0678
Epoch 7/300
Training Loss: 1.0802
Epoch 8/300
Training Loss: 1.0573
Epoch 9/300
Training Loss: 1.0541
Epoch 10/300
Training Loss: 1.0570
Epoch 11/300
Training Loss: 1.0568
Epoch 12/300
Training Loss: 1.0378
Epoch 13/300
Training Loss: 1.0470
Epoch 14/300
Training Loss: 1.0426
Epoch 15/300
Training Loss: 1.0334
Epoch 16/300
Training Loss: 1.0487
Epoch 17/300
Training Loss: 1.0294
Epoch 18/300
Training Loss: 1.0267
Epoch 19/300
Training Loss: 1.0317
Epoch 20/300
Training Loss: 1.0076
Epoch 21/300
Training Loss: 1.0033
Epoch 22/300
Training Loss: 1.0201
Epoch 23/300
Training Loss: 0.9926
Epoch 24/300
Training Loss: 1.0052
Epoch 25/300
Training Loss: 0.9993
Epoch 26/300
Training Loss: 0.9974
Epoch 27/300
Training Loss: 1.0032
Epoch 28/300
Training Loss: 0.9899
Epoch 29/300
Training Loss: 0

In [142]:
def predict(model, item_sequences, item_features, item_genres):
    """
    Predict the item with the highest probability for the given input sequences using argmax of softmax.

    Args:
    - model: The trained model.
    - item_sequences: Input item sequences (batch_size, seq_length).
    - item_features: Input item features (batch_size, feature_length).
    - item_genres: Input item genres (batch_size, genre_length).

    Returns:
    - predicted_items: A list of predicted items with the highest probability for each input sequence.
    """
    # Run the model in inference mode (not training)
    predicted_items, logits = model((item_sequences, item_features, item_genres), training=False)

    # Apply softmax to the logits to get probabilities
    probabilities = tf.nn.softmax(logits, axis=-1)

    # Get the item with the highest probability by finding the index of the maximum probability
    predicted_items = tf.argmax(probabilities, axis=-1, output_type=tf.int32)

    # Convert to numpy array for easier handling
    predicted_items = predicted_items.numpy()

    return predicted_items

def compute_recall(predicted_items, targets):
    """
    Compute the recall for the given predictions and targets.

    Args:
    - predicted_items: The predicted items (batch_size,).
    - targets: The actual next items (batch_size,).

    Returns:
    - recall: The recall metric.
    """
    # True positives: Predicted item matches the target
    true_positives = np.sum(predicted_items == targets)

    # Total relevant items (in this case, it is the number of items in the batch)
    total_items = len(targets)

    # Recall calculation
    recall = true_positives / total_items if total_items > 0 else 0
    return recall

# Initialize variables to calculate overall recall
total_true_positives = 0
total_items = 0

# Loop through training dataset and predict the most probable item
for step, batch in enumerate(train_dataset):
    item_sequences = batch['item']
    item_genres = batch['genre']
    item_features = batch['features']
    targets = batch['next_item']

    # Get the predicted item with the highest probability for each sequence in the batch
    predicted_items = predict(model, item_sequences, item_features, item_genres)

    # Compute recall for the current batch
    batch_recall = compute_recall(predicted_items, targets)
    print(f"Batch {step + 1} recall: {batch_recall:.4f}")

    # Accumulate for overall recall
    total_true_positives += np.sum(predicted_items == targets)
    total_items += len(targets)

# Calculate overall recall
overall_recall = total_true_positives / total_items if total_items > 0 else 0
print(f"Overall recall: {overall_recall:.4f}")


Batch 1 recall: 0.4688
Batch 2 recall: 0.6250
Batch 3 recall: 0.5938
Batch 4 recall: 0.4375
Batch 5 recall: 0.5000
Batch 6 recall: 0.5000
Batch 7 recall: 0.5312
Batch 8 recall: 0.6875
Batch 9 recall: 0.5312
Batch 10 recall: 0.4375
Batch 11 recall: 0.5312
Batch 12 recall: 0.6250
Batch 13 recall: 0.5625
Batch 14 recall: 0.5938
Batch 15 recall: 0.5312
Batch 16 recall: 0.5938
Batch 17 recall: 0.4375
Batch 18 recall: 0.6562
Batch 19 recall: 0.6250
Batch 20 recall: 0.4688
Batch 21 recall: 0.5625
Batch 22 recall: 0.5625
Batch 23 recall: 0.5000
Batch 24 recall: 0.6250
Batch 25 recall: 0.5625
Batch 26 recall: 0.5312
Batch 27 recall: 0.5312
Batch 28 recall: 0.3438
Batch 29 recall: 0.5938
Batch 30 recall: 0.6562
Batch 31 recall: 0.4062
Batch 32 recall: 0.6250
Batch 33 recall: 0.5312
Batch 34 recall: 0.5312
Batch 35 recall: 0.6250
Batch 36 recall: 0.5625
Batch 37 recall: 0.4688
Batch 38 recall: 0.5625
Batch 39 recall: 0.6875
Batch 40 recall: 0.5938
Batch 41 recall: 0.5625
Batch 42 recall: 0.5000
B

In [143]:
def preprocess_data_single_session(
    session,
    feature_columns,
    k=1
):
    """
    Preprocess a single session into TensorFlow dataset with split genre and features.

    Args:
    - session (list): A list of dictionaries containing session data.
    - feature_columns (list): List of numerical feature column names.
    - mean_values (dict): Mean values for numerical features for normalization.
    - std_values (dict): Std values for numerical features for normalization.
    - k (int): Minimum length of `next_item_sequences`.

    Returns:
    - tf.data.Dataset: TensorFlow dataset containing preprocessed data.
    """
    item_sequences = []
    next_item_sequences = []
    genre_sequences = []
    feature_sequences = []

    for i in range(len(session) - 1):
        # Process items
        session_item_encoded = preprocessor.preprocess_song_id(session[i]['SongID'])
        next_session_item_encoded = preprocessor.preprocess_song_id(session[i + 1]['SongID'])
        item_sequences.append(session_item_encoded)
        next_item_sequences.append(next_session_item_encoded)

        # Process genre
        genre_cleaned = preprocessor.clean_genre(session[i].get('spotify_genre', None))
        genre_sequences.append(genre_cleaned)

        # Process numerical features
        numeric_features = []
        for col in feature_columns:
            if col != 'spotify_genre':
                mean = preprocessor.mean_values.get(col, None)
                std = preprocessor.std_values.get(col, None)
                cleaned_feature = preprocessor.clean_numeric_feature(session[i].get(col, None), mean=mean, std=std)
                numeric_features.append(cleaned_feature)

        feature_sequences.append(numeric_features)

    # Filter session if next_item_sequences length is not greater than k
    if len(next_item_sequences) <= k:
        print(f"Session skipped because next item sequence length is {len(next_item_sequences)}.")
        return

    print(f"Processed session with {len(item_sequences)} items.")

    # Convert to tensors
    item_sequences = tf.stack(item_sequences, axis=0)
    next_item_sequences = tf.stack(next_item_sequences, axis=0)
    genre_sequences_tensor = tf.constant(genre_sequences, dtype=tf.int32)
    feature_sequences_tensor = tf.constant(feature_sequences, dtype=tf.float32)

    # Create TensorFlow dataset
    dataset = tf.data.Dataset.from_tensor_slices({
        'item': item_sequences,
        'genre': genre_sequences_tensor,
        'features': feature_sequences_tensor,
        'next_item': next_item_sequences
    })

    return dataset

## Cara 1

In [144]:
def predict_next(model, item_sequences, item_features, item_genres):
    """
    Predict the next item for a given input sequence.

    Args:
    - model: The trained model.
    - item_sequences: Input item sequences (batch_size, seq_length).
    - item_features: Input item features (batch_size, feature_length).
    - item_genres: Input item genres (batch_size, genre_length).

    Returns:
    - predicted_items: Predicted next item (batch_size,).
    """
    # Run inference
    _, logits = model((item_sequences, item_features, item_genres), training=False)
    # print("Logits:", logits)

    # Apply softmax to logits to get probabilities
    probabilities = tf.nn.softmax(logits, axis=-1)

    # Select the item with the highest probability
    predicted_items = tf.argmax(probabilities, axis=-1, output_type=tf.int32)

    return predicted_items.numpy()


def compute_recall_at_k(predicted_sequence, target_sequence, k):
    """
    Compute Recall@k for a given session.

    Args:
    - predicted_sequence: The predicted sequence of items.
    - target_sequence: The actual target sequence of items.
    - k: The number of items in the predicted sequence.

    Returns:
    - recall_at_k: Recall@k value.
    """
    # Count how many of the target items appear in the predicted sequence
    predicted_ids = [item['SongID'] if isinstance(item, dict) else item for item in predicted_sequence]
    target_ids = [item['SongID'] if isinstance(item, dict) else item for item in target_sequence]

    # Count how many of the target items appear in the predicted sequence
    hits = len(set(predicted_ids[:k]) & set(target_ids[:k]))
    return hits / len(target_ids[:k]) if target_ids[:k] else 0.0
    return hits / len(target_sequence[:k])


# Initialize overall metrics
total_recall = 0
session_count = 0
k = 10  # Set the value of k
print("Creating session dataset")
sessions_data = preprocessor.create_session_dataset(preprocessor.train_df)  # Use train data for training
print("Creating tensor dataset")
# Process each session in the dataset
for session in sessions_data[:100]:  # Assume train_sessions is your preprocessed session data
    if len(session) <= k:
      continue
    # print(session)
    context_length = len(session) - k
    context = session[:context_length]
    target = session[context_length:]
    dataset = preprocess_data_single_session(session, feature_columns, k=k)
    if dataset is None:
        continue
    dataset = dataset.batch(1)
    predicted_sequence = []
    current_sequence = context

    # Generate k predictions iteratively
    for batch in dataset:
        # Prepare input features for the current sequence
        item_sequences = batch['item']  # Batch of size 1
        item_features = batch['features']  # Extract corresponding features
        item_genres = batch['genre']  # Extract corresponding genres

        # Predict the next item
        predicted_item = predict_next(model, item_sequences, item_features, item_genres)
        predicted_sequence.append(predicted_item[0])  # Append the prediction

        # Update the current sequence
        current_sequence = current_sequence[1:] + [predicted_item[0]]

        if len(predicted_sequence) >= k:
            break
    ori_pred_seq = preprocessor.song_id_encoder.inverse_transform(predicted_sequence)
    print(f"Predicted Sequence: {ori_pred_seq}")
    # target_ids = preprocessor.song_id_encoder.transform([item['SongID'] if isinstance(item, dict) else item for item in target])
    print(f"True Sequence: {[item['SongID'] for item in target]}")

    # check genre
    original_item_genres = preprocessor.train_df.loc[preprocessor.train_df['SongID'].isin([item['SongID'] if isinstance(item, dict) else item for item in target]), 'spotify_genre'].values
    # print(f"Original Item Genres: {original_item_genres}")
    predicted_item_genres = preprocessor.train_df.loc[preprocessor.train_df['SongID'].isin([item['SongID'] if isinstance(item, dict) else item for item in ori_pred_seq]), 'spotify_genre'].values
    # print(f"Predicted Item Genres: {predicted_item_genres}")
    # Calculate Recall@k for the current session
    session_recall = compute_recall_at_k(predicted_sequence, target, k)
    total_recall += session_recall
    session_count += 1

    print(f"Session recall: {session_recall:.4f}")

# Calculate overall Recall@k
overall_recall_at_k = total_recall / session_count if session_count > 0 else 0
print(f"Overall Recall@{k}: {overall_recall_at_k:.4f}")


Creating session dataset
Creating tensor dataset
Overall Recall@10: 0.0000


## Cara 2

In [145]:
import tensorflow as tf

def predict_next(model, item_sequences, item_features, item_genres, k=5):
    """
    Predict the next item for a given input sequence.

    Args:
    - model: The trained model.
    - item_sequences: Input item sequences (batch_size, seq_length).
    - item_features: Input item features (batch_size, feature_length).
    - item_genres: Input item genres (batch_size, genre_length).
    - k: Number of top predictions to consider.

    Returns:
    - predicted_items: Top k predicted next items (batch_size, k).
    """
    # Run inference
    _, logits = model((item_sequences, item_features, item_genres), training=False)

    # Apply softmax to logits to get probabilities
    probabilities = tf.nn.softmax(logits, axis=-1)

    # Get the top k predictions
    top_k_values, top_k_indices = tf.nn.top_k(probabilities, k=k, sorted=True)

    return top_k_indices.numpy()  # Return top k item indices

def compute_recall_at_k(predicted_sequence, target_sequence, k):
    """
    Compute Recall@k for a given session.

    Args:
    - predicted_sequence: The predicted sequence of items.
    - target_sequence: The actual target sequence of items.
    - k: The number of top predictions to consider.

    Returns:
    - recall_at_k: Recall@k value.
    """
    # Extract 'SongID' from dictionaries if needed
    predicted_ids = [item['SongID'] if isinstance(item, dict) else item for item in predicted_sequence]
    target_ids = [item['SongID'] if isinstance(item, dict) else item for item in target_sequence]

    # Select the top k items from predicted sequence
    top_k_predicted = predicted_ids[:k]

    # Count how many of the target items appear in the top k predictions
    hits = len(set(top_k_predicted) & set(target_ids[:k]))
    return hits / len(target_ids[:k]) if target_ids[:k] else 0.0


# Initialize overall metrics
total_recall = 0
session_count = 0
k = 5  # Set the value of k

print("Creating session dataset")
sessions_data = preprocessor.create_session_dataset(preprocessor.train_df)  # Use train data for training
print("Creating tensor dataset")

# Process each session in the dataset
for session in sessions_data:  # Assume sessions_data is your preprocessed session data
    if len(session) <= k:
        continue
    print(session)
    context_length = len(session) - k
    context = session[:context_length]
    target = session[context_length:]
    dataset = preprocess_data_single_session(session, feature_columns, k=k)
    if dataset is None:
        continue
    dataset = dataset.batch(1)
    predicted_sequence = []
    current_sequence = context

    # Generate k predictions iteratively
    for batch in dataset:
        # Prepare input features for the current sequence
        item_sequences = batch['item']  # Batch of size 1
        item_features = batch['features']  # Extract corresponding features
        item_genres = batch['genre']  # Extract corresponding genres

        # Predict the top k items
        top_k_predictions = predict_next(model, item_sequences, item_features, item_genres, k=k)

        # Append the top k predictions to the sequence (only append first item from the top k)
        predicted_sequence.extend(top_k_predictions[0])  # Assuming batch size is 1

        # Update the current sequence
        current_sequence = current_sequence[1:] + [top_k_predictions[0][0]]  # Only use first predicted item for the next context

        if len(predicted_sequence) >= k:
            break

    # Calculate Recall@k for the current session
    session_recall = compute_recall_at_k(predicted_sequence, target, k)
    total_recall += session_recall
    session_count += 1

    print(f"Session recall: {session_recall:.4f}")

# Calculate overall Recall@k
overall_recall_at_k = total_recall / session_count if session_count > 0 else 0
print(f"Overall Recall@{k}: {overall_recall_at_k:.4f}")


Creating session dataset
Creating tensor dataset
[{'index_x': 62840, 'SongID': 'GulfstreamRoddie Romero & The Hub City All Stars', 'TimeStamp_Central': '1/3/2017 12:18:00 PM', 'Performer_x': 'Roddie Romero & The Hub City All Stars', 'Album': 'Gulfstream', 'Song_x': 'Gulfstream', 'TimeStamp_UTC': '2017-01-03 18:18:00', 'index_y': 13320, 'Performer_y': 'Roddie Romero & The Hub City All Stars', 'Song_y': 'Gulfstream', 'spotify_genre': "['lafayette indie']", 'spotify_track_id': '0YMdudlfDTWyn71myIMjTZ', 'spotify_track_preview_url': 'https://p.scdn.co/mp3-preview/9267c97b39723f37352e7fd6fd3e49672dd6bfc1?cid=b8d3901151d34489a160e3cf0ab1fa94', 'spotify_track_duration_ms': 368360.0, 'spotify_track_popularity': 6.0, 'spotify_track_explicit': False, 'danceability': 0.629, 'energy': 0.319, 'key': 5.0, 'loudness': -9.993, 'mode': 1.0, 'speechiness': 0.0363, 'acousticness': 0.503, 'instrumentalness': 0.000828, 'liveness': 0.111, 'valence': 0.225, 'tempo': 126.522, 'time_signature': 4.0, 'session_3_

## Cara 3

In [150]:
import tensorflow as tf

def predict_next(model, item_sequences, item_features, item_genres, k=5):
    """
    Predict the next item for a given input sequence.

    Args:
    - model: The trained model.
    - item_sequences: Input item sequences (batch_size, seq_length).
    - item_features: Input item features (batch_size, feature_length).
    - item_genres: Input item genres (batch_size, genre_length).
    - k: The number of top predictions to return.

    Returns:
    - predicted_items: Top-k predicted items (batch_size, k).
    """
    # Run inference
    _, logits = model((item_sequences, item_features, item_genres), training=False)

    # Apply softmax to logits to get probabilities
    probabilities = tf.nn.softmax(logits, axis=-1)

    # Get the top-k predicted item indices based on probabilities
    top_k_indices = tf.argsort(probabilities, axis=-1, direction='DESCENDING')[:, :k]

    return top_k_indices.numpy()


# Initialize overall metrics
total_recall = 0
session_count = 0
k = 4 # Set the value of k
print("Creating session dataset")
sessions_data = preprocessor.create_session_dataset(preprocessor.train_df)  # Use train data for training
print("Creating tensor dataset")

# Process each session in the dataset
for session in sessions_data[:100]:  # Assume train_sessions is your preprocessed session data
    if len(session) <= k:
        continue

    context_length = len(session) - k
    context = session[:context_length]
    target = session[context_length:]
    dataset = preprocess_data_single_session(session, feature_columns, k=k)
    if dataset is None:
        continue
    dataset = dataset.batch(1)
    predicted_sequence = []

    # Generate k predictions iteratively
    for batch in dataset:
        # Prepare input features for the current sequence
        item_sequences = batch['item']  # Batch of size 1
        item_features = batch['features']  # Extract corresponding features
        item_genres = batch['genre']  # Extract corresponding genres

        # Predict the top k items
        top_k_predictions = predict_next(model, item_sequences, item_features, item_genres, k)

        # Add the top-k predictions to the predicted sequence
        predicted_sequence.extend(top_k_predictions[0])  # Extend by top-k predicted items

        if len(predicted_sequence) >= k:
            break

    ori_pred_seq = preprocessor.song_id_encoder.inverse_transform(predicted_sequence)
    predicted_item_genres = preprocessor.train_df.loc[preprocessor.train_df['SongID'].isin([item['SongID'] if isinstance(item, dict) else item for item in ori_pred_seq]), 'spotify_genre'].values
    # print(f"Predicted Item Genres: {predicted_item_genres}")
    predicted_song_genre = [
      {'SongID': song_id, 'Genre': genre}
      for song_id, genre in zip(ori_pred_seq, predicted_item_genres)
    ]

    # Extract SongID and genre for the true sequence
    true_song_genre = [
        {'SongID': item['SongID'], 'Genre': preprocessor.train_df.loc[preprocessor.train_df['SongID'] == item['SongID'], 'spotify_genre'].values[0]}
        for item in target
    ]

    for true, predicted in zip(true_song_genre, predicted_song_genre):
        print(f"Predicted SongID: {predicted['SongID']}; True SongID: {true['SongID']}")
        print(f"Predicted Genre: {predicted['Genre']}; True Genre: {true['Genre']}")
        print('=====================================================================')

    # Print the predicted and true sequences with SongID and Genre
    print(f"Predicted Sequence: {predicted_song_genre}")
    print(f"True Sequence: {true_song_genre}")
    # Calculate Recall@k for the current session
    session_recall = compute_recall_at_k(predicted_sequence, target, k)
    total_recall += session_recall
    session_count += 1

    print(f"Session recall: {session_recall:.4f}")

# Calculate overall Recall@k
overall_recall_at_k = total_recall / session_count if session_count > 0 else 0
print(f"Overall Recall@{k}: {overall_recall_at_k:.4f}")


Creating session dataset
Creating tensor dataset
Processed session with 5 items.
Predicted SongID: FrancisHoundstooth; True SongID: Noam Chomsky Is A Soft RevolutionFoy Vance
Predicted Genre: ['portland indie']; True Genre: ['acoustic pop', 'british singer-songwriter', 'indie anthem-folk', 'indie folk', 'irish singer-songwriter', 'neo mellow', 'new americana', 'uk americana']
Predicted SongID: Never Say NoRobby Hunter Band; True SongID: Land Without AgeKing Washington
Predicted Genre: ['alternative rock', 'chamber pop', 'garage rock', 'indie folk', 'indie rock', 'jam band', 'melancholia', 'modern rock', 'new americana', 'roots rock', 'stomp and holler']; True Genre: []
Predicted SongID: Mannish Boy - Ruckus Roboticus RemixMuddy Waters; True SongID: Made to Break Your HeartLos Lobos
Predicted Genre: ['blues', 'blues rock', 'chicago blues', 'classic rock', 'electric blues', 'soul', 'traditional blues']; True Genre: ['roots rock']
Predicted SongID: One Big HolidayMy Morning Jacket; True S